In [2]:
%load_ext autoreload
%autoreload 2

import os, sys
from pathlib import Path
import yaml
import torch

from matplotlib import pyplot as plt

from nerfstudio.utils.eval_utils import eval_setup
# from ns_extension.utils.grouping import GroupingClassifier

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load configuration

In [5]:
# Path to the config for a trained model
load_config = '/workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/2025-07-11_171420/config.yml'
load_config = Path(load_config)

config, pipeline, checkpoint_path, step = eval_setup(load_config)

[19:19:04] Auto image downscale factor of 2                                                 ]8;id=47451;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=119991;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#484\484]8;;\

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


[19:19:29] use color only optimization with sigmoid activation                                         ]8;id=629630;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py\splatfacto.py]8;;\:]8;id=114066;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py#266\266]8;;\

Loading latest checkpoint from load_dir

✅ Done loading checkpoint from 
/workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/2025-07-11_171420/nerfstudio_models/step-00002
9999.ckpt

### Question is whether to build grouping on top of the existing data?

Initialize segmentation model

In [16]:
from ns_extension.utils.segmentation import Segmentation

segmentation = Segmentation(
    backend='mobilesamv2',
    strategy='object',
    device='cuda',
)

/opt/conda/envs/nerfstudio/lib/python3.10/site-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/RogerQi/MobileSAMV2/zipball/main" to /workspace/models/hub/main.zip
100%|██████████| 134M/134M [00:02<00:00, 55.3MB/s] 
100%|██████████| 234M/234M [00:04<00:00, 55.9MB/s] 
100%|██████████| 15.6M/15.6M [00:00<00:00, 56.2MB/s]


checkpoint_load_scucess


In [17]:
test = pipeline.datamanager.next_train(0)

[20:38:20] Caching / undistorting train images                                            ]8;id=430302;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/datamanagers/full_images_datamanager.py\full_images_datamanager.py]8;;\:]8;id=808927;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/datamanagers/full_images_datamanager.py#230\230]8;;\

Output()

In [28]:
fn = pipeline.datamanager.train_dataset.image_filenames[0]

In [29]:
# segmentation =  
from tqdm import trange
from PIL import Image
import numpy as np
from ns_extension.utils.features import resize_image

# def create_masks(image_filenames, resolution=1024):
#     """
#     Create masks for each image 
#     """

    # for i in trange(len(image_filenames), desc=f"Creating masks"):
        # Load and process image



# image = Image.open(image_filenames[i])
resolution = 1024
image = Image.open(fn)
H, W = image.height, image.width

# Prepare image for segmentation
image = resize_image(image, resolution) # Resize image to SAM resolution
image = np.asarray(image) # Convert to numpy array

# Apply segmentation masks over features
masks = segmentation.segment(image)


/opt/conda/envs/nerfstudio/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,
0: 1024x576 28 objects, 139.0ms
Speed: 4.2ms preprocess, 139.0ms inference, 44.2ms postprocess per image at shape (1, 3, 1024, 1024)


In [ ]:
masks

torch.Size([28, 1024, 576])

In [15]:
pipeline.datamanager.train_dataset.metadata

{'depth_filenames': None,
 'depth_unit_scale_factor': 0.001,
 'mask_color': None,
 'points3D_xyz': tensor([[ 0.5881,  0.2354,  0.3443],
         [ 0.5153,  0.2859,  0.1964],
         [ 0.5296,  0.2778,  0.2392],
         ...,
         [ 0.4155,  0.4354,  0.4631],
         [-0.0926,  0.2761,  0.3994],
         [ 0.4062,  0.4324,  0.4728]]),
 'points3D_rgb': tensor([[174, 173, 178],
         [ 20,  10,   6],
         [104,  97,  96],
         ...,
         [ 77,  81,  59],
         [ 52,  54,  45],
         [ 41,  50,  32]], dtype=torch.uint8),
 'feature_type': 'samclip',
 'feature_dims': {'dinov2': torch.Size([384, 57, 32]),
  'samclip': torch.Size([768, 113, 64])}}